In [ ]:
import tensorflow as tf
from tensorflow.keras import models, optimizers, regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import matplotlib.pyplot as plt 

In [ ]:
model1 = tf.keras.Sequential()

model1.add(Conv2D(filters=16,kernel_size=3,padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), input_shape=(256,180,3)))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=2))
model1.add(Dropout(0.35))

model1.add(Conv2D(filters=32,kernel_size=3,padding='same', activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)))
model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=2))
model1.add(Dropout(0.35))

model1.add(Conv2D(filters=64,kernel_size=3,padding='same', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4), activation='relu'))
model1.add(MaxPooling2D(pool_size=2))
model1.add(BatchNormalization())
model1.add(Dropout(0.35))

model1.add(Flatten())
model1.add(Dense(256))
model1.add(BatchNormalization())
model1.add(Dense(1, activation='sigmoid'))
model1.summary()

In [ ]:
model2 = model1
model3 = model1

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.3,
    horizontal_flip = False
)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator1 = train_datagen.flow_from_directory('../input/tarros-dataset-final-for-real/DATASET_improved/1_front/train',
                                 target_size = (256,180),
                                 batch_size = 4,
                                 class_mode = "binary")

validation_generator1 = test_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/1_front/validation",
                                 target_size = (256,180),
                                 batch_size = 1,
                                 class_mode = "binary")

In [ ]:
train_generator2 = train_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/2_back/train",
                                 target_size = (256,180),
                                 batch_size = 4,
                                 class_mode = "binary")

validation_generator2 = test_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/2_back/validation",
                                 target_size = (256,180),
                                 batch_size = 1,
                                 class_mode = "binary")

In [ ]:
train_generator3 = train_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/3_up/train",
                                 target_size = (256,180),
                                 batch_size = 4,
                                 class_mode = "binary")

validation_generator3 = test_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/3_up/validation",
                                 target_size = (256,180),
                                 batch_size = 1,
                                 class_mode = "binary")

In [ ]:
checkpoint1 = ModelCheckpoint('tarros_1.hdf5', monitor = "val_accuracy", verbose = 1, save_best_only = True)
checkpoint2 = ModelCheckpoint('tarros_2.hdf5', monitor = "val_accuracy", verbose = 1, save_best_only = True)
checkpoint3 = ModelCheckpoint('tarros_3.hdf5', monitor = "val_accuracy", verbose = 1, save_best_only = True)

In [ ]:
model1.compile(loss = "binary_crossentropy",
             optimizer = 'rmsprop',
             metrics = ["accuracy"])

In [ ]:
model2.compile(loss = "binary_crossentropy",
             optimizer = 'rmsprop',
             metrics = ["accuracy"])

In [ ]:
model3.compile(loss = "binary_crossentropy",
             optimizer='rmsprop',
             metrics = ["accuracy"])

In [ ]:
hist1 = model1.fit(train_generator1,
                steps_per_epoch = 448//4,
                epochs = 1,
                validation_data = validation_generator1,
                validation_steps = 56//1,
                callbacks = [checkpoint1])

In [ ]:
hist2 = model2.fit(train_generator2,
                steps_per_epoch = 448//4,
                epochs = 70,
                validation_data = validation_generator2,
                validation_steps = 56//1,
                callbacks = [checkpoint2])

In [ ]:
hist3 = model3.fit(train_generator3,
                steps_per_epoch = 448//4,
                epochs = 70,
                validation_data = validation_generator3,
                validation_steps = 56//1,
                callbacks = [checkpoint3])

In [ ]:
plt.plot(hist1.history["accuracy"], label = "Train")
plt.plot(hist1.history["val_accuracy"], label = "Validation")
plt.legend()
plt.title('Front-Side performance')
plt.show()
plt.savefig('tarros_1.png')

In [ ]:
plt.plot(hist2.history["accuracy"], label = "Train")
plt.plot(hist2.history["val_accuracy"], label = "Validation")
plt.legend()
plt.title('Back-Side performance')
plt.show()
plt.savefig('tarros_2.png')

In [ ]:
plt.plot(hist3.history["accuracy"], label = "Train")
plt.plot(hist3.history["val_accuracy"], label = "Validation")
plt.legend()
plt.title('Up-Side performance')
plt.show()
plt.savefig('tarros_3.png')

In [ ]:
test_model1 = load_model('./tarros_1.hdf5')
test_model2 = load_model('./tarros_2.hdf5')
test_model3 = load_model('./tarros_3.hdf5')

In [ ]:
test_generator1 = test_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/1_front/test",
                                 target_size = (256,180),
                                 batch_size = 1,
                                 class_mode = "binary")

In [ ]:
test_generator2 = test_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/2_back/test",
                                  target_size = (256,180),
                                  batch_size = 1,
                                  class_mode = "binary")

In [ ]:
test_generator3 = test_datagen.flow_from_directory("../input/tarros-dataset-final-for-real/DATASET_improved/3_up/test",
                                  target_size = (256,180),
                                  batch_size = 1,
                                  class_mode = "binary")

In [ ]:
test_model1.evaluate(test_generator1)
test_model2.evaluate(test_generator2)
test_model3.evaluate(test_generator3)